# Regras de Associação

***


## Análise do conjunto de dados

Regras de associação são algoritmos que extraem conjuntos de itens frequentes em datasets que cada instância é um conjunto de itens. Vamos visualizar o que isso significa.

Vamos observar o dataset de compras de um supermercado. Segue o link para mais informações [dataset](https://archive.ics.uci.edu/dataset/352/online+retail)

In [1]:
# Carregando o dataset

import pandas as pd
import numpy as np

df = pd.read_csv("https://raw.githubusercontent.com/Francimaria/monitoria-ml/main/online_retail.csv")


Vamos visualizar uma pequena parte desse dataset.

In [2]:
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom


Aqui temos diversas informações sobre compras em um supermercado:

- (_InvoiceNo_) o número da fatura, identificador de uma compra
- (_StockCode_) o código de certo produto no estoque
- (_Description_) a descrição do produto
- (_Quantity_) a quantidade de produtos que foi comprada
- (_InvoiceDate_) o dia da compra
- (_UnitPrice_) o preço por unidade
- (_CustomerID_) o id do consumidor
- (_Country_) o país de venda

In [3]:
# vamos olhar todos os países existentes
print(df["Country"].unique())
# print(df["Country"].value_counts())

# dos países, vamos escolher apenas as vendas feitas no Reino Unido, apenas pelo fato de existirem mais vendas no dataset
country = "United Kingdom"
sales = df[df["Country"] == country]

['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'EIRE' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Austria' 'Israel' 'Finland' 'Bahrain' 'Greece' 'Hong Kong'
 'Singapore' 'Lebanon' 'United Arab Emirates' 'Saudi Arabia'
 'Czech Republic' 'Canada' 'Unspecified' 'Brazil' 'USA'
 'European Community' 'Malta' 'RSA']


In [4]:
# antes de utilizar nosso dataset, precisamos fazer alguns pre-processamentos:
# remover todas as instãncias que não possuem _InvoiceNo_
filtered_sales = sales.dropna(axis=0, subset=["InvoiceNo"])
# transformar em strings
filtered_sales["InvoiceNo"] = filtered_sales["InvoiceNo"].astype("str")
# remover instancias que contenham "C" no _InvoiceNo_, representando instancias que não possuem essa feature
filtered_sales = filtered_sales[~filtered_sales["InvoiceNo"].str.contains("C")]

# remover espaços desnecessários no começo e fim de cada _Description_
filtered_sales["Description"] = filtered_sales["Description"].str.strip()
filtered_sales["Description"] = filtered_sales["Description"].astype("str")


Como o dataset a ser avaliado deverá consistir em conjuntos de itens, vamos transformar cada compra em um conjunto de itens. Note que um conjunto não possui informação sobre quantidade de elementos repetidos. Portanto, a coluna com a contagem de cada item deve ser removida, indicando apenas a presença daquele item em uma compra.

In [36]:
sales_set = filtered_sales.groupby(['InvoiceNo', 'Description'])["Quantity"].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
sales_set.head()

Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
sales_set[["POSTAGE", "DOTCOM POSTAGE"]]

Description,POSTAGE,DOTCOM POSTAGE
InvoiceNo,,
536365,0.0,0.0
536366,0.0,0.0
536367,0.0,0.0
536368,0.0,0.0
536369,0.0,0.0
...,...,...
581585,0.0,0.0
581586,0.0,0.0
A563185,0.0,0.0


In [37]:
# existem algumas vendas que tiveram algum tipo de problema, vamos remove-las a partir no index 4057

#vamos reduzir o número de colunas para 1500
sales_set = sales_set.iloc[:, :1500]
print(sales_set.shape)

# também estão descritos o tipo de postagem, se pela internet ou não, vamos remove-los
sales_set = sales_set.drop("POSTAGE", axis=1, errors="ignore")
sales_set = sales_set.drop("DOTCOM POSTAGE", axis=1, errors="ignore")

# nosso dataset final
sales_set.head()

(20122, 1500)


Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,GLASS CHALICE GREEN SMALL,GLASS CLOCHE LARGE,GLASS CLOCHE SMALL,GLASS HEART T-LIGHT HOLDER,GLASS JAR DAISY FRESH COTTON WOOL,GLASS JAR DIGESTIVE BISCUITS,GLASS JAR ENGLISH CONFECTIONERY,GLASS JAR KINGS CHOICE,GLASS JAR MARMALADE,GLASS JAR PEACOCK BATH SALTS
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
# agora, vamos transformar o dataset de uma contagem de itens, para apenas conjuntos
# faremos isso transformando toda contagem para uma presença ou não do item

count_to_set = lambda x: x > 0 # aqui se uma venda possui pelo menos 1 item, afime True, caso contrário, False
sales_set = sales_set.applymap(count_to_set)

print(sales_set.shape)
sales_set.head()

(20122, 1499)


Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,GLASS CHALICE GREEN SMALL,GLASS CLOCHE LARGE,GLASS CLOCHE SMALL,GLASS HEART T-LIGHT HOLDER,GLASS JAR DAISY FRESH COTTON WOOL,GLASS JAR DIGESTIVE BISCUITS,GLASS JAR ENGLISH CONFECTIONERY,GLASS JAR KINGS CHOICE,GLASS JAR MARMALADE,GLASS JAR PEACOCK BATH SALTS
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536366,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536367,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536368,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536369,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Após esse paço de pre-processamento, conseguimos extrair quais itens são comprados em conjunto com maior frequência para todas as vendas.

## Apriori

Inicialmente, vamos utilizar o algoritmo a priori. Nele precisamos indicar qual a repetição mínima necessária de repetição que buscamos. Por exemplo, se quisermos apenas as repetições que aconteçam pelo menos $n\%$ das vezes.

O algoritmo funciona por criar todas as combinações possíveis de conjuntos e então checar suas frequências.

In [39]:
from mlxtend.frequent_patterns import apriori

# use_colnames retorna o itemset como nomes ao invés de indices das colunas
frequency_set = apriori(sales_set, min_support=0.01, use_colnames=True)
print(frequency_set)

#Unable to allocate 9.89 GiB for an array with shape (263901, 2, 20122) and data type bool

      support                                           itemsets
0    0.013567                           (10 COLOUR SPACEBOY PEN)
1    0.011977                  (12 MESSAGE CARDS WITH ENVELOPES)
2    0.016251                    (12 PENCIL SMALL TUBE WOODLAND)
3    0.016847              (12 PENCILS SMALL TUBE RED RETROSPOT)
4    0.016748                      (12 PENCILS SMALL TUBE SKULL)
..        ...                                                ...
253  0.010337  (FELTCRAFT CUSHION OWL, FELTCRAFT CUSHION RABBIT)
254  0.011480  (FELTCRAFT PRINCESS CHARLOTTE DOLL, FELTCRAFT ...
255  0.026886  (GARDENERS KNEELING PAD CUP OF TEA, GARDENERS ...
256  0.011778  (ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...
257  0.012722  (ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...

[258 rows x 2 columns]


In [40]:
# perceba que temos muitas conjuntos com apenas um elemento
# vamos filtrar esses conjuntos que tem apenas um elemento

def filter_set_lenght(input_set, lenght=2):
    input_set['set_lenght'] = input_set['itemsets'].apply(lambda x: len(x))
    new_set = input_set[input_set['set_lenght'] >= lenght]
    new_set.reset_index(inplace=True, drop=True)
    return new_set
    
combo_set = filter_set_lenght(frequency_set)
print(combo_set)

     support                                           itemsets  set_lenght
0   0.012375  (72 SWEETHEART FAIRY CAKE CASES, 60 TEATIME FA...           2
1   0.012375  (ALARM CLOCK BAKELIKE CHOCOLATE, ALARM CLOCK B...           2
2   0.012623  (ALARM CLOCK BAKELIKE CHOCOLATE, ALARM CLOCK B...           2
3   0.015207  (ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...           2
4   0.012027  (ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...           2
5   0.017394  (ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...           2
6   0.027979  (ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...           2
7   0.012325  (ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...           2
8   0.016996  (ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...           2
9   0.013170  (ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...           2
10  0.019531  (ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...           2
11  0.014015  (BAKING SET 9 PIECE RETROSPOT, BAKING SET SPAC...           2
12  0.011331

In [41]:
# escolhendo a combinação de itens com maior repetição
def get_highest_support(input_set):
    instance = input_set.iloc[input_set["support"].idxmax()]
    return instance

# e criando uma função auxiliar para mostrar as estatísticas do nosso combo a partir do nosso conjunto de vendas
def print_combo_stats(combo, sales):
    print("itens: %s"%(str(tuple(combo["itemsets"]))))
    print("frequencia %.2f%%" %(float(combo["support"])*100))
    print("vendas totais do combo: %d"%(float(combo["support"]) * sales.shape[0]))

combo = get_highest_support(combo_set)
print_combo_stats(combo, sales_set)

itens: ('ALARM CLOCK BAKELIKE GREEN', 'ALARM CLOCK BAKELIKE RED')
frequencia 2.80%
vendas totais do combo: 563


## FP-Growth

Ao contrário do algoritmo a priori, FP-Growth não precisa criar todas os conjuntos de combinações. Para datasets em que a quantidade de combinações é muito grande, ele possui uma vantagem sobre seu tempo de execução.

In [42]:
# caso o import fpgrowth falhe, descomente a linha abaixo
# !pip install mlxtend -U

from mlxtend.frequent_patterns.fpgrowth import fpgrowth

# use_colnames retorna o itemset como nomes ao invés de indices das colunas
frequency_set = fpgrowth(sales_set, min_support=0.01, use_colnames=True)
print(frequency_set)

      support                                           itemsets
0    0.013319                   (CREAM CUPID HEARTS COAT HANGER)
1    0.068134                    (ASSORTED COLOUR BIRD ORNAMENT)
2    0.027930                              (DOORMAT NEW ENGLAND)
3    0.020525                (FELTCRAFT PRINCESS CHARLOTTE DOLL)
4    0.011778                   (BOX OF VINTAGE ALPHABET BLOCKS)
..        ...                                                ...
253  0.013865  (CHARLOTTE BAG SUKI DESIGN, CHARLOTTE BAG APPL...
254  0.011331  (CHARLOTTE BAG APPLES DESIGN, CHARLOTTE BAG PI...
255  0.010735  (CHARLOTTE BAG APPLES DESIGN, CHARLOTTE BAG VI...
256  0.011530  (CHARLOTTE BAG SUKI DESIGN, CHARLOTTE BAG VINT...
257  0.026886  (GARDENERS KNEELING PAD CUP OF TEA, GARDENERS ...

[258 rows x 2 columns]


In [43]:
combo_set = filter_set_lenght(frequency_set)
combo = get_highest_support(combo_set)
print_combo_stats(combo, sales_set)

itens: ('ALARM CLOCK BAKELIKE GREEN', 'ALARM CLOCK BAKELIKE RED')
frequencia 2.80%
vendas totais do combo: 563


Temos o mesmo resultado, mas vamos avaliar o tempo de execução entre cada algoritmo...

In [44]:
from time import time
t0 = time()
frequency_set = apriori(sales_set, min_support=0.01, use_colnames=True)
t1 = time()
frequency_set = fpgrowth(sales_set, min_support=0.01, use_colnames=True)
t2 = time()

print("APriori(t_delta): %f" %(t1-t0))
print("FP-Growth(t_delta): %f" %(t2-t1))

APriori(t_delta): 2.119570
FP-Growth(t_delta): 0.926785
